# Calculate crop area and share for 3 specific ARISE sites within Bengo based on the KAZA Land Cover 2020 dataset

These values can be used as a baseline to compare to the values derived from the predictions from random forest and OpenMapFlow.

## Setup

In [1]:
import os
import rasterio
import rasterio.mask
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
def load_and_preprocess_roi(roi_path, roi_region, roi_names):
    
    roi = gpd.read_file(os.path.join(roi_path, roi_region + '.shp'))
    roi.columns = roi.columns.str.lower()
    roi = roi[['name', 'geometry']]
    roi = roi[roi['name'].isin(roi_names)]
    roi = roi.dissolve()
    roi['area_km2'] = roi['geometry'].to_crs({'proj':'cea'}).area[0] / 10**6
    roi = roi.to_crs(epsg=32734)
    
    return roi

In [3]:
def clip_raster(raster_file_path, region_geodataframe):

    with rasterio.open(raster_file_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, [region_geodataframe.loc[0]['geometry']], crop=True)
        out_meta = src.meta

        out_meta.update(
            {
                'height': out_image.shape[1],
                'width': out_image.shape[2],
                'transform': out_transform,
            }
        )
        
        out_file_path = raster_file_path.replace(raster_file_path.split('/')[-1], '')
        out_file_name = 'temp.tif'

        with rasterio.open(os.path.join(out_file_path, out_file_name), 'w', **out_meta) as dest:
            dest.write(out_image)
    
    return out_transform

In [4]:
def extract_data_from_raster(raster_file_path):
    
    raster = rasterio.open(raster_file_path)    
    masked_array = raster.read(1, masked=True)
    
    no_data = raster.nodata    
    data = masked_array.data
    
    row, col = np.where(data != no_data)
    land_cover_class = np.extract(data != no_data, data)
    
    dataframe = pd.DataFrame({'col': col, 'row': row, 'land_cover_class': land_cover_class})
    
    return dataframe

In [5]:
def calculate_crop_area_and_share(roi_path, roi_region, roi_names, raster_file_path, raster_path):
    
    # load and preprocess roi
    roi = load_and_preprocess_roi(roi_path, roi_region, roi_names)
    
    # clip raster
    out_transform = clip_raster(raster_file_path, roi)
    
    # extract data
    df = extract_data_from_raster(os.path.join(raster_path, 'temp.tif'))
    
    # get amount of crop pixels in roi
    crop_pixels = df[df['land_cover_class'].isin([40])].shape[0]
    
    # calculate crop share
    crop_share = crop_pixels / df.shape[0]
    
    # calculate crop area
    crop_area = roi.loc[0]['area_km2'] * crop_share
    
    print(f'ROI region: {roi_region}\nROI names: {roi_names}\nCrop area: {crop_area:.2f} km²\nCrop share: {crop_share:.2%}\n')

## Load KAZA Land Cover 2020 dataset

In [6]:
raster_path = '../raw_data/kaza_land_cover_2020'
raster_file = 'Land_Cover_KAZA_2021_TFCA.tif'
raster_file_path = os.path.join(raster_path, raster_file)
kaza_land_cover_2020 = rasterio.open(raster_file_path)

## Calculate crop area and share for regions of interest

In [7]:
roi_path = '../raw_data/arise_sites'

In [8]:
%%time
calculate_crop_area_and_share(roi_path, 'NAM_sites_bengo', ['Sikunga'], raster_file_path, raster_path)

ROI region: NAM_sites_bengo
ROI names: ['Sikunga']
Crop area: 0.42 km²
Crop share: 0.15%

CPU times: user 645 ms, sys: 264 ms, total: 909 ms
Wall time: 912 ms


In [9]:
%%time
calculate_crop_area_and_share(roi_path, 'ZAM_sites_bengo', ['Nyawa'], raster_file_path, raster_path)

ROI region: ZAM_sites_bengo
ROI names: ['Nyawa']
Crop area: 154.28 km²
Crop share: 36.71%

CPU times: user 795 ms, sys: 227 ms, total: 1.02 s
Wall time: 1.03 s


In [10]:
%%time
calculate_crop_area_and_share(roi_path, 'ZIM_sites_bengo', ['Kachechete', 'Chikandakubi', 'Chidobe', 'Nemananga'], raster_file_path, raster_path)

ROI region: ZIM_sites_bengo
ROI names: ['Kachechete', 'Chikandakubi', 'Chidobe', 'Nemananga']
Crop area: 177.64 km²
Crop share: 24.66%

CPU times: user 1.11 s, sys: 325 ms, total: 1.43 s
Wall time: 1.44 s
